In [ ]:
# this is a simulation of CA1/CA3 'place field' populations that project onto a single LS neuron. 
# david tingley 2018

%matplotlib inline 
from pyNN.neuron import *
import matplotlib.pyplot as plt
import numpy as np
import scipy.io as sio
import scipy.signal as sig   
from scipy.stats.mstats import zscore
from IPython import display
import time
import neo as neo

def get_cmap(n, name='jet'):
    '''Returns a function that maps each index in 0, 1, ..., n-1 to a distinct 
    RGB color; the keyword argument name must be a standard mpl colormap name.'''
    return plt.cm.get_cmap(name, n)

def getRate(spkCount,ls_voltages):
    spk = (ls_voltages.segments[-1].spiketrains[0])
    h = np.histogram(spk,3500,range=(0,runtime))
    spkCount = spkCount + h[0]
    box = np.append(np.zeros((85,1)),np.append(sig.boxcar(30)/30, np.zeros((85,1))))
    spkCount = np.convolve(spkCount[0,:], box, mode='same')
    return spkCount

def plot_spiketrains(segment,offset):
    c=get_cmap(len(segment.spiketrains))
    for spiketrain in segment.spiketrains:
        spk_id = spiketrain.annotations['source_index']
        y = np.ones_like(spiketrain) * spk_id
        plt.plot(spiketrain, y, '.',color=c(spk_id))
        # plt.ylabel(segment.name)
        plt.setp(plt.gca().get_xticklabels(), visible=True)

def plot_phaseclouds(segment,phaseJitters):
    c=get_cmap(len(segment.spiketrains))
    for spiketrain in segment.spiketrains:
        spk_id = spiketrain.annotations['source_index']
        # if spk_id == 10 or spk_id == 50: # comment to plot single cell across trials
        for spk in np.arange(0,len(spiketrain),1):
            plt.plot(spiketrain[spk],phaseJitters[int(round(100*spiketrain[spk]))],'.',color=c(spk_id))
        # plt.ylabel(segment.name)
        plt.setp(plt.gca().get_xticklabels(), visible=True)

cellparams = {
        'gbar_Na'   : 20.0,
        'gbar_K'    : 6.0,
        'g_leak'    : 0.01,
        'cm'        : 0.2,
        'v_offset'  : -63.0,
        'e_rev_Na'  : 50.0,
        'e_rev_K'   : -90.0,
        'e_rev_leak': -65.0,
        'e_rev_E'   : -6.0,  #
        'e_rev_I'   : -71.0, #
        'tau_syn_E' : 1.0,
        'tau_syn_I' : 2.0,
        'i_offset'  : 0.0,
    }


plt.ion()
fig = plt.figure(figsize=(12,12))

# free parameters
numcells = 50
thetaFreq = 8
interval = 6 # we want fields to span/tile the 'space', assuming a constant running velocity
offsets = 2 # number of theta offsets. this mimics the transverse axis, CA1-CA3. 
lsInputThresh = .0001 * 100 / offsets  # controls net excitation per unit time
runtime = 3500 
numIterations = 200
# also see 'fieldInput' and 'thetaInput' below..

# initialize and load data
spkCount = np.zeros((runtime,numIterations))
data = sio.loadmat('assemblyWeights.mat')
ca1Weights = data['ca1'] 
ca3Weights = data['ca3'] 
ca1Weights = np.interp(np.linspace(0,200,numcells),np.linspace(0,200,200),ca1Weights[:,0]) # interp to # of cells
ca3Weights = np.interp(np.linspace(0,200,numcells),np.linspace(0,200,200),ca3Weights[:,0]) #

for iteration in range(0,numIterations,1):
    for fieldInput in [.033 * 100/interval]:# [.033]: #np.arange(.03,.045,.005): # field input (arange is the range of values tested)
        for thetaInput in [.01]: #np.arange(.008,.015,.003): # theta input (arange is the range of values tested)
            setup(timestep=0.01, min_delay=1.0)
            phaseJitter = np.random.uniform(0,360) # make sure the position/theta phase relationship is random across trials
            fieldJitter = np.random.uniform(0,250) # add a little noise to individual fields (not necessary, just more realistic)
            
            if iteration == 0: # set up the populations and connection weights on first trial
                standard_model = StaticSynapse(weight=0,delay=7.0);
                LS = Population(1, HH_cond_exp(**cellparams), label='LS')
                CA123 = []
                CA123_connections = []
                for p in range(0,offsets,1):
                    CA123.append(Population(numcells, HH_cond_exp, {'e_rev_I': -75}, label='HPC'))
                    CA123_connections.append(Projection(CA123[p],LS,AllToAllConnector(allow_self_connections=False),standard_model))
                    CA123[p].record('spikes')
                for p in range(0,offsets,1):
                    if p == 0:
                        newWeights = ca1Weights 
                    else:
                        newWeights = ca3Weights 
                    CA123_connections[p].setWeights(newWeights)

            # trial specific stuff to set up
            synTheta = [] # make theta input for this trial
            for p in range(0,offsets,1):
                synTheta.append(ACSource(start=0, stop=runtime, amplitude=thetaInput, offset=thetaInput,
                                    frequency=thetaFreq, phase=phaseJitter + np.linspace(0,180,offsets)[p]))

            LS.record('spikes')
            LS.record('v')
#             for p in range(0,offsets,1):
#                 CA123[p].inject(synTheta[p])

            # let's make some place/time fields...
            for i in range(0, numcells, 1):
                fieldStart = 500 + int(i*interval+fieldJitter+np.random.uniform(-3,3)) # add some jitter across both trials and cells..
                fieldStop = int(fieldStart+7)
                fieldStop = int(fieldStart+len(np.arange(0, fieldInput,(fieldInput)/(fieldStop-fieldStart))))
                synfieldInput = StepCurrentSource(times=np.arange(fieldStart, fieldStop+1,1),amplitudes=np.append(np.arange(0, fieldInput,(fieldInput)/(fieldStop-fieldStart)),0))
                for p in range(0,offsets,1):
                    CA123[p][i].inject(synfieldInput)
                if iteration == 0:
                    plt.subplot(2,3,1)
                    c=get_cmap(numcells)
                    plt.plot(synfieldInput.times,np.array(synfieldInput.amplitudes)+i,color=c(i))
                    plt.plot((0,runtime),(0,0),'k')
                    plt.xlim((0, runtime))
                    plt.axis('off')
                    
            # reverse replay
            for i in range(0, numcells, 1):
                fieldStart = 1500 + int((numcells-i)*interval+fieldJitter+np.random.uniform(-3,3)) # add some jitter across both trials and cells..
                fieldStop = int(fieldStart+7)
                fieldStop = int(fieldStart+len(np.arange(0, fieldInput,(fieldInput)/(fieldStop-fieldStart))))
                synfieldInput = StepCurrentSource(times=np.arange(fieldStart, fieldStop+1,1),amplitudes=np.append(np.arange(0, fieldInput,(fieldInput)/(fieldStop-fieldStart)),0))
                for p in range(0,offsets,1):
                    CA123[p][i].inject(synfieldInput)
                if iteration == 0:
                    plt.subplot(2,3,2)
                    c=get_cmap(numcells)
                    plt.plot(synfieldInput.times,np.array(synfieldInput.amplitudes)+i,color=c(i))
                    plt.plot((0,runtime),(0,0),'k')
                    plt.xlim((0, runtime))
                    plt.axis('off')
                    
            # random event
            order = np.random.permutation(numcells)
            for i in range(0, numcells, 1):
                fieldStart = 2500 + int(order[i]*interval+fieldJitter+np.random.uniform(-3,3)) # add some jitter across both trials and cells..
                fieldStop = int(fieldStart+7)
                fieldStop = int(fieldStart+len(np.arange(0, fieldInput,(fieldInput)/(fieldStop-fieldStart))))
                synfieldInput = StepCurrentSource(times=np.arange(fieldStart, fieldStop+1,1),amplitudes=np.append(np.arange(0, fieldInput,(fieldInput)/(fieldStop-fieldStart)),0))
                for p in range(0,offsets,1):
                    CA123[p][i].inject(synfieldInput)
                if iteration == 0:
                    plt.subplot(2,3,3)
                    c=get_cmap(numcells)
                    plt.plot(synfieldInput.times,np.array(synfieldInput.amplitudes)+i,color=c(i))
                    plt.plot((0,runtime),(0,0),'k')
                    plt.xlim((0, runtime))
                    plt.axis('off')
                    
            # run one trial
            run(runtime)

            # collect those data
            CA123_voltages=[]
            for p in range(0,offsets,1):
                CA123_voltages.append(CA123[p].get_data())
            ls_voltages = LS.get_data()
  
            spkCount[:,iteration] = getRate(np.zeros((1,runtime)),ls_voltages)
            # plot some stuffs
            if np.mod(iteration,1) == 0 or iteration == 0:
                plt.subplot(2,2,3)
                plt.cla()
                plt.imshow(np.transpose(spkCount),aspect='auto')
                plt.plot((500, 500),(0, iteration),'r')
                plt.plot((1500, 1500),(0, iteration),'r')
                plt.plot((2500, 2500),(0, iteration),'r')
                
                plt.subplot(2,2,4)
                plt.cla()
                plt.plot(np.mean(spkCount,1))
                plt.plot((500, 500),(0, max(np.mean(spkCount,1))),'r')
                plt.plot((1500, 1500),(0, max(np.mean(spkCount,1))),'r')
                plt.plot((2500, 2500),(0, max(np.mean(spkCount,1))),'r')  
                print('iteration: ' + str(iteration))
                display.display(plt.gcf())
                display.clear_output(wait=True)
    # uncomment below if you would like to save data
#     io = neo.io.NeoMatlabIO(filename='data/dynamic_ls_output_' + str(iteration) + '.mat')
#     LS.write_data(io)
#     sio.savemat('data/dynamic_trial_' + str(iteration) + '_theta.mat',mdict={'theta': synTheta[0].amplitudes})
    # reset sim
    reset(annotations={"iteration" : iteration})

In [ ]:
plt.figure()
plt.figure(figsize=(12,12))
plt.imshow(np.transpose(spkCount),aspect='auto')
plt.plot((500, 500),(0, iteration),'r')
plt.plot((1500, 1500),(0, iteration),'r')
plt.plot((2500, 2500),(0, iteration),'r')

In [ ]:
plt.figure(figsize=(12,12))
plt.plot(np.mean(spkCount,1))
plt.plot((500, 500),(0, .06),'r')
plt.plot((1500, 1500),(0, .06),'r')
plt.plot((2500, 2500),(0, .06),'r')